# Get the dataset

In [1]:
import numpy as np
import config

dataset_image = np.load(config.x_path, mmap_mode="r")
dataset_label = np.load(config.y_path, mmap_mode="r")
print(dataset_image.shape, dataset_label.shape)
assert(dataset_image.shape[0] == dataset_label.shape[0])


(10, 3, 224, 224) (10,)


# Boilerplate Code

In [2]:
from sklearn.model_selection import KFold
import config

kfold = KFold(config.kfold_nsplits, shuffle=True, random_state=config.seed)


In [3]:
from typing import Tuple
import torch

class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, label: np.ndarray) -> None:
        assert(image.shape[0] == label.shape[0])
        self.image = image
        self.label = label
    
    def __iter__(self):
        for x, y in zip(self.image, self.label):
            yield x, y
    
    def __len__(self):
        return self.image.shape[0]
    
    def __getitem__(self, idx: int):
        return self.image[idx], self.label[idx]


# Model

In [4]:
from torch import nn
from torchvision.models import vgg16, VGG16_Weights


def get_model() -> nn.Module:
    model = vgg16(weights=VGG16_Weights.DEFAULT, progress=True)
    in_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(in_features, 2)
    return model


In [5]:
model = get_model()

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

model = model.to(device)
criterion = criterion.to(device)


# Train

In [1]:
from tqdm import tqdm
def train(train_idx: np.ndarray) -> Tuple(float, float):
    print("TRAINING")
    train = Data(dataset_image[train_idx], dataset_label[train_idx])
    train_dataloader = torch.utils.data.DataLoader(
        train, batch_size=config.batch_size)
    print("MADE DATALOADER")
    total_loss_train = 0
    total_accumulate_train = 0
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)
        output = model(image)
        print("OUT: ", output)
        print("LABEL: ", label)

        batch_loss = criterion(output, label)

        total_loss_train += batch_loss

        accumulate = (output.argmax(dim=1) == label).sum()
        print(accumulate, type(accumulate))
        total_accumulate_train += accumulate

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()

    total_loss_train = total_loss_train.item()
    total_accumulate_train = total_accumulate_train.item()
    return (total_loss_train, total_accumulate_train)


NameError: name 'np' is not defined

In [7]:
def judge(judge_idx: np.ndarray) -> Tuple(float, float):
    judge = Data(dataset_image[judge_idx], dataset_label[judge_idx])
    judge_dataloader = torch.utils.data.DataLoader(
        judge, batch_size=config.batch_size)

    total_loss_judge = 0
    total_accumulate_judge = 0
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)

            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss

            accumulate = (output.argmax(dim=1) == label).sum()
            print(accumulate, type(accumulate))
            total_accumulate_judge += accumulate

    total_loss_judge = total_loss_judge.item()
    total_accumulate_judge = total_accumulate_judge.item()
    return total_loss_judge, total_accumulate_judge

In [8]:

min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(kfold.split(dataset_label)):
    print(epoch)
    total_loss_train, total_accumulate_train = train(train_idx)
    total_loss_judge, total_accumulate_judge = judge(judge_idx)

    print(
        f'''Epochs: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}'''
        
    )
    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), "simplemodel.pt")
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")

0
TRAINING
MADE DATALOADER


  0%|          | 0/1 [00:00<?, ?it/s]

OUT:  tensor([[ 0.8105,  0.0932],
        [ 0.6063,  1.0488],
        [ 1.0895,  0.3594],
        [-0.0539,  0.0170],
        [ 0.8855,  0.4818],
        [-1.0410, -0.0190],
        [ 0.6168,  0.2529],
        [-0.0567, -0.1889],
        [ 0.5724,  0.2531]], device='cuda:0', grad_fn=<AddmmBackward0>)
LABEL:  tensor([0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8)
tensor(4, device='cuda:0') <class 'torch.Tensor'>


100%|██████████| 1/1 [00:00<00:00, 30.30it/s]


tensor(1, device='cuda:0') <class 'torch.Tensor'>
Epochs: 1 
        | Train Loss: 0.079
        | Train Accuracy: 0.444
        | Val Loss: 0.000
        | Val Accuracy: 1.000
Save model because val loss improve loss 0.000
1
TRAINING
MADE DATALOADER


  0%|          | 0/1 [00:00<?, ?it/s]

OUT:  tensor([[-32.0443,  33.5971],
        [-38.9766,  42.6384],
        [-23.7156,  40.5637],
        [-27.0355,  27.9077],
        [-18.0297,  31.6772],
        [-37.2963,  32.7003],
        [-37.7067,  34.8429],
        [-27.3638,  32.7682],
        [-27.6980,  35.8973]], device='cuda:0', grad_fn=<AddmmBackward0>)
LABEL:  tensor([0, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8)
tensor(8, device='cuda:0') <class 'torch.Tensor'>


  0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 2.39 GiB already allocated; 0 bytes free; 3.09 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF